In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

%matplotlib inline

In [2]:
token_values = pd.read_csv('data/hw3.csv')
token_values.info
#Null Values exist

<bound method DataFrame.info of                        ts       open       high        low      close  \
0     2021-12-02 14:00:00    22.4150    22.4913    22.0816    22.3516   
1     2021-12-02 23:00:00     4.8043     4.8043     4.7426     4.7806   
2     2021-12-10 15:00:00   182.4900        NaN   175.2100   175.8600   
3     2021-12-10 18:00:00  3978.4300  3989.7400  3932.0000  3972.3400   
4     2021-12-08 21:00:00   193.3240   194.2420   192.5640   193.1540   
...                   ...        ...        ...        ...        ...   
2662  2021-12-03 12:00:00   243.5040   244.4700   243.0960   243.7920   
2663  2021-12-03 07:00:00   231.1080        NaN   231.1080   234.5980   
2664  2021-12-10 23:00:00     3.4600     3.4800     3.4100     3.4200   
2665  2021-12-03 23:00:00  4215.0000  4249.7200  4183.2300  4198.8300   
2666  2021-12-06 07:00:00     1.0012     1.0012     1.0011     1.0011   

            volume  volumeUSD token chain  
0     3.150215e+04        NaN   UNI   ETH  
1  

In [4]:
token_values['token'].value_counts()
#Token incorrect formatting

BTC                                   323
COMP                                  322
CRV                                   318
SOL                                   318
USDT                                  314
UNI                                   312
AAVE                                  312
ETH                                   298
<span name="tokenName">UNI</span>      30
<span name="tokenName">ETH</span>      28
<span name="tokenName">USDT</span>     19
<span name="tokenName">CRV</span>      17
<span name="tokenName">SOL</span>      16
<span name="tokenName">AAVE</span>     16
<span name="tokenName">BTC</span>      14
<span name="tokenName">COMP</span>     10
Name: token, dtype: int64

In [5]:
token_values.duplicated().sum()
#Exists duplicates

307

In [6]:
#fixing token names
token_values['token']=token_values['token'].str.replace('<span name="tokenName">', '')
token_values['token']=token_values['token'].str.replace('</span>', '')

token_values['token'].value_counts()

UNI     342
BTC     337
CRV     335
SOL     334
USDT    333
COMP    332
AAVE    328
ETH     326
Name: token, dtype: int64

In [7]:
#drop duplicates
token_values = token_values.drop_duplicates()
token_values['token'].value_counts()

UNI     295
CRV     295
SOL     295
ETH     295
AAVE    295
USDT    295
COMP    295
BTC     295
Name: token, dtype: int64

In [8]:
#missing close price
#sort by token and ts then forward filled the data for High and Low
#Then used heuristic method in class to find the close price
token_values.sort_values(['token','ts'],inplace=True)
token_values['low'].fillna(method='ffill',inplace=True)
token_values['high'].fillna(method='ffill',inplace=True)
token_values.loc[token_values['close'].isnull(), 'close'] = 0.5 * (token_values.loc[token_values['close'].isnull(), 'high'] + token_values.loc[token_values['close'].isnull(), 'low'])
token_values.loc[token_values.close / token_values.open >= 2, 'close'] = 0.5 * (token_values.loc[token_values.close / token_values.open >= 2, 'high'] + token_values.loc[token_values.close / token_values.open >= 2, 'low'])

token_values

/Users/alvinyu/opt/anaconda3/envs/mfe-preprogram/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/Users/alvinyu/opt/anaconda3/envs/mfe-preprogram/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/Users/alvinyu/opt/anaconda3/envs/mfe-preprogram/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,ts,open,high,low,close,volume,volumeUSD,token,chain
2199,2021-12-01 00:00:00,257.1020,260.7750,255.3450,257.0780,2730.299,NaN,AAVE,ETH
932,2021-12-01 01:00:00,257.1490,266.2490,255.2700,264.8160,5752.541,NaN,AAVE,ETH
2582,2021-12-01 02:00:00,264.7550,266.1870,262.5970,263.1250,1559.330,NaN,AAVE,ETH
363,2021-12-01 03:00:00,263.1840,266.1870,262.5970,266.2570,1647.398,NaN,AAVE,ETH
1758,2021-12-01 04:00:00,266.3840,267.0240,264.2840,265.5020,829.993,NaN,AAVE,ETH
...,...,...,...,...,...,...,...,...,...
1814,2021-12-13 02:00:00,1.0005,1.0006,1.0004,1.0004,7133859.470,NaN,USDT,USDT
614,2021-12-13 03:00:00,1.0004,1.0006,1.0004,1.0006,4337884.530,NaN,USDT,USDT
1992,2021-12-13 04:00:00,1.0006,1.0006,1.0004,1.0006,2928892.910,NaN,USDT,USDT
859,2021-12-13 05:00:00,1.0006,1.0006,1.0005,1.0006,2178755.170,NaN,USDT,USDT


In [9]:
print(np.isnan(token_values['close']).any())

False


In [10]:
#calculating volume by usd
token_values['volumeUSD']=token_values['volume']*token_values['close']
token_values

/var/folders/r2/mw9wckgx4_sg3qqs0gt_pjww0000gn/T/ipykernel_8455/2760284360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  token_values['volumeUSD']=token_values['volume']*token_values['close']


,ts,open,high,low,close,volume,volumeUSD,token,chain
2199,2021-12-01 00:00:00,257.1020,260.7750,255.3450,257.0780,2730.299,7.018998e+05,AAVE,ETH
932,2021-12-01 01:00:00,257.1490,266.2490,255.2700,264.8160,5752.541,1.523365e+06,AAVE,ETH
2582,2021-12-01 02:00:00,264.7550,266.1870,262.5970,263.1250,1559.330,4.102987e+05,AAVE,ETH
363,2021-12-01 03:00:00,263.1840,266.1870,262.5970,266.2570,1647.398,4.386312e+05,AAVE,ETH
1758,2021-12-01 04:00:00,266.3840,267.0240,264.2840,265.5020,829.993,2.203648e+05,AAVE,ETH
...,...,...,...,...,...,...,...,...,...
1814,2021-12-13 02:00:00,1.0005,1.0006,1.0004,1.0004,7133859.470,7.136713e+06,USDT,USDT
614,2021-12-13 03:00:00,1.0004,1.0006,1.0004,1.0006,4337884.530,4.340487e+06,USDT,USDT
1992,2021-12-13 04:00:00,1.0006,1.0006,1.0004,1.0006,2928892.910,2.930650e+06,USDT,USDT
859,2021-12-13 05:00:00,1.0006,1.0006,1.0005,1.0006,2178755.170,2.180062e+06,USDT,USDT


In [11]:
token_values.groupby('chain')['volumeUSD'].sum().to_frame()

,volumeUSD
chain,
BTC,1.085555e+10
ETH,1.356072e+10
SOL,4.012098e+09
USDT,1.005307e+09


In [13]:
token_values.loc[token_values['chain'] == 'USDT']

,ts,open,high,low,close,volume,volumeUSD,token,chain
1382,2021-12-01 00:00:00,1.0008,1.0009,1.0008,1.0008,1588401.81,1.589673e+06,USDT,USDT
1681,2021-12-01 01:00:00,1.0008,1.0009,1.0008,1.0009,3429521.89,3.432608e+06,USDT,USDT
973,2021-12-01 02:00:00,1.0009,1.0009,1.0008,1.0008,2136351.34,2.138060e+06,USDT,USDT
587,2021-12-01 03:00:00,1.0009,1.0010,1.0008,1.0008,2974174.19,2.976554e+06,USDT,USDT
1756,2021-12-01 04:00:00,1.0009,1.0010,1.0008,1.0008,4970508.62,4.974485e+06,USDT,USDT
...,...,...,...,...,...,...,...,...,...
1814,2021-12-13 02:00:00,1.0005,1.0006,1.0004,1.0004,7133859.47,7.136713e+06,USDT,USDT
614,2021-12-13 03:00:00,1.0004,1.0006,1.0004,1.0006,4337884.53,4.340487e+06,USDT,USDT
1992,2021-12-13 04:00:00,1.0006,1.0006,1.0004,1.0006,2928892.91,2.930650e+06,USDT,USDT
859,2021-12-13 05:00:00,1.0006,1.0006,1.0005,1.0006,2178755.17,2.180062e+06,USDT,USDT


ts           2021-12-13 06:00:00
open                      1.0054
high                       1.025
low                       1.0017
close                      1.013
volume               32828498.67
volumeUSD         33255269.15271
token                       USDT
chain                       USDT
dtype: object